In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from scipy import stats
import tensorflow
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pickle
import json

# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

import warnings
warnings.filterwarnings('ignore')

In [1]:
def LSTM_model(timestamp=6):
        
        csv_files = pickle.load(open('Tech_data_path.pkl', 'rb'))
        
        
        def X_Y(df, timestamp):
            X, Y = [], []
            for i in range(0, len(df) - timestamp - 1):
                X.append(df[i:(i + timestamp), :])
                Y.append(df[i + timestamp, :])

            if len(df) >= timestamp:
                X.append(df[-timestamp:, :])
                Y.append(df[-1, :])

            return np.array(X), np.array(Y)
        
        def get_confidence_interval(errors, confidence=0.95):
            mean_error = np.mean(errors)
            standard_error = np.std(errors, ddof=1) / np.sqrt(len(errors))

            t_score = np.abs(stats.t.ppf((1 + confidence) / 2, len(errors) - 1))

            lower_bound = mean_error - t_score * standard_error
            upper_bound = mean_error + t_score * standard_error
            
             # Inverse transform to get prices in the original scale
            lower_bound = scaler.inverse_transform([[0, 0, 0, lower_bound]])[0, -1]
            upper_bound = scaler.inverse_transform([[0, 0, 0, upper_bound]])[0, -1]

            return round(lower_bound), round(upper_bound)

        def get_recommendation(model, xtest, ytest, scaler, last_day_price):
            pred = model.predict(xtest)

            last_sequence = xtest[-1]
            last_sequence = last_sequence.reshape(1, last_sequence.shape[0], last_sequence.shape[1])

            next_day_pred = model.predict(last_sequence)
            next_day_pred_original_scale = scaler.inverse_transform(next_day_pred)
            next_day_closing_price = next_day_pred_original_scale[0, -1]

            errors = np.abs(ytest - pred)
            confidence_interval = get_confidence_interval(errors)

            accuracy = r2_score(ytest, pred)
            current_price = last_day_price[-1]

            recommendation = ((next_day_closing_price - current_price) / current_price) * 100

            return current_price, recommendation, accuracy, confidence_interval

        result_data = []
        for file_path in csv_files:
            
            input_filename = os.path.basename(file_path).split(".")[0]
            
            df = pd.read_csv(file_path)
            df['Datetime'] = pd.to_datetime(df['Datetime']).dt.date

            original_data = df[['Open', 'High', 'Low',"Close"]].values
            scaler = MinMaxScaler(feature_range=(0, 1))
            scaled_data = scaler.fit_transform(original_data)

            train_size = int(len(scaled_data) * 0.70)
            xtrain, ytrain = X_Y(scaled_data[0:train_size], timestamp)
            xtest, ytest = X_Y(scaled_data[train_size:], timestamp)

            xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 4)
            xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 4)

            model = Sequential()
            model.add(LSTM(50, return_sequences=True, input_shape=(xtrain.shape[1], 4)))
            model.add(LSTM(50, return_sequences=True))
            model.add(LSTM(50))
            model.add(Dense(4))
            model.compile(loss='mse', optimizer='adam')

            history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=1, batch_size=64, verbose=1)

            current_price, recommendation, accuracy, confidence_interval  = get_recommendation(model, xtest, ytest, scaler, df['Close'].values)
            result_data.append({
                "stock_ticker": input_filename,
                "current_price": round(current_price,2),
                "recommendation":  f"{round(recommendation,2)}%",
                "accuracy": f"{round(accuracy,2)*100}%",
                "confidence_interval": confidence_interval
            })
        # json.dump(result_data, open('Output/result_data.json', 'w'))
        print(result_data)
        return result_data


In [4]:
LSTM_model()

1/1 [==============================] - 0s 217ms/step
[{'stock_ticker': 'HDFC_data', 'current_price': 2733.0, 'recommendation': '5.31%', 'accuracy': '-147.0%', 'confidence_interval': (2331, 2336)}, {'stock_ticker': 'LNT_data', 'current_price': 2565.0, 'recommendation': '-0.11%', 'accuracy': '79.0%', 'confidence_interval': (1785, 1790)}, {'stock_ticker': 'TCS_data', 'current_price': 3397.5, 'recommendation': '4.33%', 'accuracy': '-108.0%', 'confidence_interval': (3063, 3070)}, {'stock_ticker': 'TITAN_data', 'current_price': 3029.95, 'recommendation': '-5.63%', 'accuracy': '78.0%', 'confidence_interval': (2344, 2352)}]


[{'stock_ticker': 'HDFC_data',
  'current_price': 2733.0,
  'recommendation': '5.31%',
  'accuracy': '-147.0%',
  'confidence_interval': (2331, 2336)},
 {'stock_ticker': 'LNT_data',
  'current_price': 2565.0,
  'recommendation': '-0.11%',
  'accuracy': '79.0%',
  'confidence_interval': (1785, 1790)},
 {'stock_ticker': 'TCS_data',
  'current_price': 3397.5,
  'recommendation': '4.33%',
  'accuracy': '-108.0%',
  'confidence_interval': (3063, 3070)},
 {'stock_ticker': 'TITAN_data',
  'current_price': 3029.95,
  'recommendation': '-5.63%',
  'accuracy': '78.0%',
  'confidence_interval': (2344, 2352)}]

In [5]:
result_data = json.load(open('Output/result_data.json', 'r'))

In [7]:
result_data[0]

{'stock_ticker': 'HDFC_data',
 'current_price': 2733.0,
 'recommendation': '0.72%',
 'accuracy': '84.0%',
 'confidence_interval': [2254, 2258]}

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from scipy import stats
import tensorflow
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pickle
import json